In [243]:
import cv2
import os
import numpy as np
from random import shuffle
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [244]:
# TRAIN_DIR = 'J:/SONGS/LAP DOWNLOAD/GEETA/Compressed/Data of saudia/Data of saudia'
TRAIN_DIR = 'C:/Users/DELL/SkyDrive/Documents/data2/train_data'
# TEST_DIR = 'J:/SONGS/LAP DOWNLOAD/GEETA/Compressed/Data of saudia/tes'
TEST_DIR = 'C:/Users/DELL/SkyDrive/Documents/data2/test_data'
IMG_SIZE = 50
LR = 1e-3
 
 
'''Setting up the model which will help with tensorflow models'''
MODEL_NAME = 'Autistic-{}-{}.model'.format(LR, '6conv-basic')

In [245]:
def label_img(img):
    word_label = img.split('.')#[-3]
    s=word_label[0]
 # DIY One hot encoder
    if 'happy' in s : 
        return [1, 0, 0, 0]
    elif 'sad' in  s: 
        return [0, 1, 0, 0]
    elif 'angry' in  s: 
        return [0, 0, 1, 0] 
    elif 'normal' in  s: 
        return [0, 0, 1, 1] 

In [246]:
def create_train_data():
    # Creating an empty list where we should the store the training data
    # after a little preprocessing of the data
    training_data = []
 
    # tqdm is only used for interactive loading
    # loading the training data
    for img in tqdm(os.listdir(TRAIN_DIR)):
 
        # labeling the images
        label = label_img(img)
 
        path = os.path.join(TRAIN_DIR, img)
 
        # loading the image from the path and then converting them into
        # greyscale for easier covnet prob
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
 
        # resizing the image for processing them in the covnet
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
 
        # final step-forming the training data list with numpy array of the images
        training_data.append([np.array(img), np.array(label)])
 
    # shuffling of the training data to preserve the random state of our data
    shuffle(training_data)
 
    # saving our trained data for further uses if required
    np.save('train_data.npy', training_data)
    return training_data
 

In [247]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('.')[0]
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img), img_num])
         
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data
 

In [248]:
'''Running the training and the testing in the dataset for our model'''
train_data = create_train_data()
test_data = process_test_data()
 

100%|███████████████████████████████████████| 105/105 [00:00<00:00, 105.24it/s]


In [210]:
# test_data.shape

In [249]:
train_data[0][0]
type(train_data)
len(train_data)

413

In [250]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
 
import tensorflow as tf
tf.reset_default_graph()
convnet = input_data(shape =[None, IMG_SIZE, IMG_SIZE, 1], name ='input')
 
convnet = conv_2d(convnet, 32, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = conv_2d(convnet, 64, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = conv_2d(convnet, 128, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = conv_2d(convnet, 64, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = conv_2d(convnet, 32, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)
 
convnet = fully_connected(convnet, 1024, activation ='relu')
convnet = dropout(convnet, 0.3)
 
convnet = fully_connected(convnet, 4, activation ='softmax')
convnet = regression(convnet, optimizer ='adam', learning_rate = LR,
      loss ='categorical_crossentropy', name ='targets')
 
model = tflearn.DNN(convnet, tensorboard_dir ='log')

In [251]:
# random classifier model implementation

from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
# clf = RandomForestClassifier(max_depth=2, random_state=0)
clf = RandomForestClassifier(n_estimators=100)
RandomX = np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
RandomY = [i[1] for i in train_data]
# RandomX=RandomX.reshape(27,2500)
RandomX=RandomX.reshape(len(train_data),IMG_SIZE*IMG_SIZE)
print("RandomX shape",RandomX.shape)

#  svm y editing
svm_y = []
for val in RandomY:
    if np.argmax(val) == 0:
#         str_label ='Happy'
        svm_y.append(0)
    elif np.argmax(val) == 1:
#         str_label ='sad'
        svm_y.append(1)
    elif np.argmax(val) == 2:
#         str_label ='Angry'
        svm_y.append(2)
    elif np.argmax(val) == 3:
#         str_label ='normal'
        svm_y.append(3)


RandomY_numpy_array = np.array(RandomY)
# type(RandomX)
# print((RandomY))
# print((RandomY.shape))
print((RandomY_numpy_array.shape))
# print((RandomY.ndim))
clf.fit(RandomX,RandomY)
s_v_m = svm.SVC(gamma=0.001,C=100)
s_v_m.fit(RandomX,svm_y)

RandomX shape (413, 2500)
(413, 4)


SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [252]:
len(train_data)
int(len(train_data) * .3)

123

In [253]:
# Splitting the testing data and training data
split_value=int(len(train_data) * .3) # 30 /70 split 
train = train_data[:-split_value]
test = train_data[-split_value:]


In [254]:
# X-Features & Y-Labels
 
X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = [i[1] for i in train]
test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = [i[1] for i in test]
# print("test_y",len(test_y))

In [273]:
model_details = model.fit({'input': X}, {'targets': Y}, n_epoch = 10, 
    validation_set =({'input': test_x}, {'targets': test_y}), 
    snapshot_step = 2000, show_metric = True, run_id = MODEL_NAME)
model.save(MODEL_NAME)

Training Step: 1074  | total loss: 0.58143 | time: 0.811s
| Adam | epoch: 215 | loss: 0.58143 - acc: 0.9447 -- iter: 256/290
Training Step: 1075  | total loss: 0.56581 | time: 2.091s
| Adam | epoch: 215 | loss: 0.56581 - acc: 0.9424 | val_loss: 2.58254 - val_acc: 0.6585 -- iter: 290/290
--
INFO:tensorflow:C:\Users\DELL\Autistic-0.001-6conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.


In [272]:
'''Testing the data'''
from sklearn import metrics
import matplotlib.pyplot as plt
# if you need to create the data:
# test_data = process_test_data()
# if you already have some saved:
test_data = np.load('test_data.npy')
 
# fig = plt.figure()
Label_OF_test_data = [] 
Predicted_label = []
Predicted_label_RandomForest = []
b = True


for num, data in enumerate(test_data):
    # cat: [1, 0]
    # dog: [0, 1]
     
    img_num = data[1]
    img_data = data[0]
#     print(img_num)
    if 'happy' in img_num : 
        Label_OF_test_data.append(0)
    elif 'sad' in  img_num: 
        Label_OF_test_data.append(1)
    elif 'angry' in  img_num: 
        Label_OF_test_data.append(2)
    elif 'normal' in  img_num: 
        Label_OF_test_data.append(3)
        
    
#     y = fig.add_subplot(4, 5, num + 1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
 
    # model_out = model.predict([data])[0]
#     print(data.shape)
#     print(data.ndim)
    
    dataRC=data.reshape(1,IMG_SIZE*IMG_SIZE)
#     print(dataRC.shape)
#     print(dataRC.ndim)
#     if b:
#         break
    model_out = model.predict([data])[0]
#     print(np.argmax(model_out))
    if np.argmax(model_out) == 0:
#         str_label ='Happy'
        Predicted_label.append(0)
    elif np.argmax(model_out) == 1:
#         str_label ='sad'
        Predicted_label.append(1)
    elif np.argmax(model_out) == 2:
#         str_label ='Angry'
        Predicted_label.append(2)
    elif np.argmax(model_out) == 3:
#         str_label ='normal'
        Predicted_label.append(3)
    
        

score=metrics.accuracy_score(Label_OF_test_data, Predicted_label)
print(Label_OF_test_data)
print(Predicted_label)
print('Accuracy cnn : {}'.format(score))


[2, 3, 1, 3, 2, 3, 0, 0, 2, 3, 1, 2, 1, 3, 1, 1, 3, 3, 3, 1, 1, 1, 2, 1, 3, 0, 0, 2, 0, 2, 0, 2, 2, 1, 0, 0, 0, 0, 1, 2, 3, 3, 1, 1, 2, 1, 0, 3, 2, 1, 0, 3, 2, 0, 2, 1, 1, 2, 0, 0, 1, 1, 1, 2, 0, 2, 0, 3, 0, 2, 0, 3, 2, 1, 2, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 0, 1, 1, 1, 3, 0, 0, 2, 1, 3, 2, 0, 0, 2, 3, 3, 3, 3]
[2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 1, 2, 0, 2, 1, 1, 2, 2, 1, 2, 2, 2, 0, 0, 2, 0, 2, 1, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 1, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 0]
Accuracy cnn : 0.42857142857142855


In [260]:
# RANDOM CLASSIFIER PREDICTION

test_Xrc = np.array([i[0] for i in test_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
print(test_Xrc.ndim)
print(test_Xrc.shape)
test_Xrc=test_Xrc.reshape(len(test_Xrc),IMG_SIZE*IMG_SIZE)
print(test_Xrc.ndim)
print(test_Xrc.shape)
# *************************IMPLEMENTING rANDOM fOREST MODEL ******************************************
pred_rc=clf.predict(test_Xrc)
# print(pred_rc)
final_pred_rc=[]
for val in pred_rc:
    if np.argmax(val) == 0:
#         str_label ='Happy'
        final_pred_rc.append(0)
    elif np.argmax(val) == 1:
#         str_label ='sad'
        final_pred_rc.append(1)
    elif np.argmax(val) == 2:
#         str_label ='Angry'
        final_pred_rc.append(2)
    elif np.argmax(val) == 3:
#         str_label ='normal'
        final_pred_rc.append(3)
# RC ACCURACY CHECK

score=metrics.accuracy_score(Label_OF_test_data, final_pred_rc)
print(Label_OF_test_data)
print(final_pred_rc)
print('Accuracy RANDOM CLASSIFIER: {}'.format(score))

4
(105, 50, 50, 1)
2
(105, 2500)
[2, 3, 1, 3, 2, 3, 0, 0, 2, 3, 1, 2, 1, 3, 1, 1, 3, 3, 3, 1, 1, 1, 2, 1, 3, 0, 0, 2, 0, 2, 0, 2, 2, 1, 0, 0, 0, 0, 1, 2, 3, 3, 1, 1, 2, 1, 0, 3, 2, 1, 0, 3, 2, 0, 2, 1, 1, 2, 0, 0, 1, 1, 1, 2, 0, 2, 0, 3, 0, 2, 0, 3, 2, 1, 2, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 0, 1, 1, 1, 3, 0, 0, 2, 1, 3, 2, 0, 0, 2, 3, 3, 3, 3]
[2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 2, 2, 0, 2, 0, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2]
Accuracy RANDOM CLASSIFIER: 0.4


In [169]:
#  implemrnting SVM 
pred_svm=[]
pred_svm=s_v_m.predict(test_Xrc)
# print(pred_rc)
#  accuracy scheck svm

score=metrics.accuracy_score(Label_OF_test_data, pred_svm)
print(Label_OF_test_data)
print(pred_svm)
print('Accuracy SVM: {}'.format(score))

[3, 2, 0, 1, 3, 1, 0, 1, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 2, 1, 1, 3, 2, 2, 1, 1, 3, 3, 2, 3, 0, 3, 1, 3, 3, 3, 0, 0, 3, 1, 2, 3, 1, 0, 3, 1, 1, 0, 0, 1, 0, 2, 0, 3, 1, 2, 3, 2, 0, 2, 2, 2, 2, 2, 3, 2, 3, 0, 3, 2, 2, 2, 3, 1, 3, 0, 2, 0, 2, 2, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 1, 1, 2, 0, 1, 0, 1, 3]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
Accuracy SVM: 0.23809523809523808


In [170]:
# implementing Logistic regression
from sklearn.linear_model import LogisticRegression

# all parameters not specified are set to their defaults# all pa 
# default solver is incredibly slow thats why we change it
# solver = 'lbfgs'
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(RandomX, svm_y)
pred_logReg=logisticRegr.predict(test_Xrc)
score=metrics.accuracy_score(Label_OF_test_data, pred_logReg)
print(Label_OF_test_data)
print(pred_logReg)
print('Accuracy LogisticRegression : {}'.format(score))

[3, 2, 0, 1, 3, 1, 0, 1, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 2, 1, 1, 3, 2, 2, 1, 1, 3, 3, 2, 3, 0, 3, 1, 3, 3, 3, 0, 0, 3, 1, 2, 3, 1, 0, 3, 1, 1, 0, 0, 1, 0, 2, 0, 3, 1, 2, 3, 2, 0, 2, 2, 2, 2, 2, 3, 2, 3, 0, 3, 2, 2, 2, 3, 1, 3, 0, 2, 0, 2, 2, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 1, 1, 2, 0, 1, 0, 1, 3]
[2 2 0 2 2 2 0 0 2 0 0 2 2 2 2 2 0 2 0 0 1 0 2 2 1 2 2 2 2 0 2 2 2 2 2 0 1
 2 2 0 2 0 0 2 2 0 2 2 0 0 2 2 2 0 0 0 2 0 0 1 2 2 2 0 2 2 2 2 2 2 2 2 0 0
 2 2 2 2 2 0 0 2 2 2 2 2 0 0 2 2 0 0 2 0 0 2 2 0 2 2 2 1 2 2 2]
Accuracy LogisticRegression : 0.41904761904761906


In [171]:
# KNN IMPLEMENTATION
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(RandomX, svm_y)
pred_knn=neigh.predict(test_Xrc)
score=metrics.accuracy_score(Label_OF_test_data, pred_knn)
print(Label_OF_test_data)
print(pred_knn)
print('Accuracy KNN : {}'.format(score))

[3, 2, 0, 1, 3, 1, 0, 1, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 2, 1, 1, 3, 2, 2, 1, 1, 3, 3, 2, 3, 0, 3, 1, 3, 3, 3, 0, 0, 3, 1, 2, 3, 1, 0, 3, 1, 1, 0, 0, 1, 0, 2, 0, 3, 1, 2, 3, 2, 0, 2, 2, 2, 2, 2, 3, 2, 3, 0, 3, 2, 2, 2, 3, 1, 3, 0, 2, 0, 2, 2, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 1, 1, 2, 0, 1, 0, 1, 3]
[2 2 0 2 0 2 0 0 0 0 0 2 2 2 2 2 2 2 2 0 2 2 0 2 2 2 2 2 2 1 2 2 2 2 2 0 2
 2 2 0 0 0 0 2 2 2 0 0 2 0 2 2 1 0 0 0 2 0 2 1 2 2 2 0 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 0 0 2 2 2 2 2 0 0 2 2 2 0 2 2 0 2 2 2 2 2 2 2 2 2 2]
Accuracy KNN : 0.4


In [172]:
# LinearDiscriminantAnalysis IMPLEMENTATION
clf = LinearDiscriminantAnalysis()
clf.fit(RandomX, svm_y)
pred_LinearDiscriminantAnalysis=clf.predict(test_Xrc)
score=metrics.accuracy_score(Label_OF_test_data, pred_LinearDiscriminantAnalysis)
print(Label_OF_test_data)
print(pred_LinearDiscriminantAnalysis)
print('Accuracy LinearDiscriminantAnalysis : {}'.format(score))


C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


[3, 2, 0, 1, 3, 1, 0, 1, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 2, 1, 1, 3, 2, 2, 1, 1, 3, 3, 2, 3, 0, 3, 1, 3, 3, 3, 0, 0, 3, 1, 2, 3, 1, 0, 3, 1, 1, 0, 0, 1, 0, 2, 0, 3, 1, 2, 3, 2, 0, 2, 2, 2, 2, 2, 3, 2, 3, 0, 3, 2, 2, 2, 3, 1, 3, 0, 2, 0, 2, 2, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 1, 1, 2, 0, 1, 0, 1, 3]
[2 2 0 2 2 2 0 0 2 0 0 2 2 0 2 2 0 0 0 2 0 0 2 0 2 2 2 2 2 2 2 0 2 2 2 0 2
 2 2 0 2 0 2 2 2 2 2 2 2 0 2 1 2 0 0 0 2 0 0 1 2 2 2 2 2 2 2 2 2 2 2 1 2 0
 2 2 2 2 2 0 0 2 2 2 2 2 0 2 2 2 0 0 2 0 0 2 2 2 2 2 2 1 2 2 2]
Accuracy LinearDiscriminantAnalysis : 0.37142857142857144


In [173]:
# GaussianNB IMPLEMENTATION
clf = GaussianNB()
clf.fit(RandomX, svm_y)
pred_GaussianNB=clf.predict(test_Xrc)
score=metrics.accuracy_score(Label_OF_test_data, pred_GaussianNB)
print(Label_OF_test_data)
print(pred_GaussianNB)
print('Accuracy GaussianNB : {}'.format(score))

[3, 2, 0, 1, 3, 1, 0, 1, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 2, 1, 1, 3, 2, 2, 1, 1, 3, 3, 2, 3, 0, 3, 1, 3, 3, 3, 0, 0, 3, 1, 2, 3, 1, 0, 3, 1, 1, 0, 0, 1, 0, 2, 0, 3, 1, 2, 3, 2, 0, 2, 2, 2, 2, 2, 3, 2, 3, 0, 3, 2, 2, 2, 3, 1, 3, 0, 2, 0, 2, 2, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 1, 1, 2, 0, 1, 0, 1, 3]
[2 2 2 2 0 2 0 0 2 0 0 1 2 2 0 1 2 1 2 1 1 0 2 2 2 1 1 2 2 2 2 0 2 1 2 0 1
 1 1 2 2 0 0 2 2 2 2 2 2 2 2 1 1 0 2 0 2 0 0 1 2 2 2 0 2 2 2 1 2 2 2 1 1 0
 2 2 2 2 2 1 0 2 1 2 2 0 2 0 0 2 2 0 2 0 0 2 2 2 2 2 1 1 2 2 1]
Accuracy GaussianNB : 0.41904761904761906


In [174]:
# DecisionTreeClassifier 
clf = DecisionTreeClassifier(random_state=0)
clf.fit(RandomX, svm_y)
pred_DecisionTreeClassifier=clf.predict(test_Xrc)
score=metrics.accuracy_score(Label_OF_test_data, pred_DecisionTreeClassifier)
print(Label_OF_test_data)
print(pred_DecisionTreeClassifier)
print('Accuracy DecisionTreeClassifier : {}'.format(score))

[3, 2, 0, 1, 3, 1, 0, 1, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 2, 1, 1, 3, 2, 2, 1, 1, 3, 3, 2, 3, 0, 3, 1, 3, 3, 3, 0, 0, 3, 1, 2, 3, 1, 0, 3, 1, 1, 0, 0, 1, 0, 2, 0, 3, 1, 2, 3, 2, 0, 2, 2, 2, 2, 2, 3, 2, 3, 0, 3, 2, 2, 2, 3, 1, 3, 0, 2, 0, 2, 2, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 1, 1, 2, 0, 1, 0, 1, 3]
[2 2 2 1 0 1 2 2 2 2 0 1 0 2 2 1 2 2 2 0 2 2 2 0 1 1 2 2 2 0 2 0 0 2 2 0 2
 0 2 0 2 0 0 2 2 2 2 0 2 2 2 1 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 0 2
 2 2 2 2 2 2 2 2 1 2 2 2 0 2 0 2 0 2 2 2 0 2 0 2 2 2 0 1 2 2 2]
Accuracy DecisionTreeClassifier : 0.3523809523809524


# This part of code is implementing PCA and ml algo

In [175]:
# *********************** PCA + lOGISTIC REGRESSION *****************

from sklearn.decomposition import PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd

In [176]:
RandomX.shape

(413, 2500)

In [177]:
# Pca_X = []
# Pca_Y = []
Pca_train_X = []
Pca_test__X=[]
Pca_train_X=RandomX
Pca_test_X=test_Xrc
print(Pca_train_X.shape)
print(len(Pca_train_X))
print(Pca_test_X.shape)
print(len(Pca_test_X))

(413, 2500)
413
(105, 2500)
105


In [178]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit on training set only.
scaler.fit(Pca_train_X)

# Apply transform to both the training set and the test set.
Pca_train_X = scaler.transform(Pca_train_X)
Pca_test_X = scaler.transform(Pca_test_X)

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [179]:
from sklearn.decomposition import PCA
pca = PCA(.95)
pca.fit(Pca_train_X)

PCA(copy=True, iterated_power='auto', n_components=0.95, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [180]:
pca.n_components_

148

In [181]:
Pca_train_X = pca.transform(Pca_train_X)
Pca_test_X = pca.transform(Pca_test_X)

In [182]:
from sklearn.linear_model import LogisticRegression

# all parameters not specified are set to their defaults# all pa 
# default solver is incredibly slow thats why we change it
# solver = 'lbfgs'
logisticRegr = LogisticRegression(solver = 'lbfgs')
logisticRegr.fit(Pca_train_X, svm_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False)

In [184]:
pred_logReg=logisticRegr.predict(Pca_test_X)

In [185]:
score=metrics.accuracy_score(Label_OF_test_data, pred_logReg)
print(Label_OF_test_data)
print(pred_logReg)
print('Accuracy PCA + LogisticRegression : {}'.format(score))

[3, 2, 0, 1, 3, 1, 0, 1, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 2, 1, 1, 3, 2, 2, 1, 1, 3, 3, 2, 3, 0, 3, 1, 3, 3, 3, 0, 0, 3, 1, 2, 3, 1, 0, 3, 1, 1, 0, 0, 1, 0, 2, 0, 3, 1, 2, 3, 2, 0, 2, 2, 2, 2, 2, 3, 2, 3, 0, 3, 2, 2, 2, 3, 1, 3, 0, 2, 0, 2, 2, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 1, 1, 2, 0, 1, 0, 1, 3]
[2 2 0 2 0 2 0 0 2 0 0 2 2 2 2 2 0 2 0 2 0 0 2 2 2 2 2 2 2 0 2 2 2 1 2 0 1
 2 2 0 0 0 0 2 2 0 2 0 0 0 2 1 2 0 0 0 2 0 2 1 2 2 2 0 2 2 2 2 2 2 2 2 0 0
 1 2 2 2 2 0 0 2 2 2 2 2 0 0 2 2 0 0 2 0 0 0 2 2 2 2 2 1 2 2 0]
Accuracy PCA + LogisticRegression : 0.4


In [186]:
# **************************** PCA + SVM *****************
s_v_m.fit(Pca_train_X,svm_y)
pred_svm_pca=s_v_m.predict(Pca_test_X)
# print(pred_rc)
#  accuracy scheck svm

score=metrics.accuracy_score(Label_OF_test_data, pred_svm_pca)
print(Label_OF_test_data)
print(pred_svm_pca)
print('Accuracy PCA + SVM: {}'.format(score))

[3, 2, 0, 1, 3, 1, 0, 1, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 2, 1, 1, 3, 2, 2, 1, 1, 3, 3, 2, 3, 0, 3, 1, 3, 3, 3, 0, 0, 3, 1, 2, 3, 1, 0, 3, 1, 1, 0, 0, 1, 0, 2, 0, 3, 1, 2, 3, 2, 0, 2, 2, 2, 2, 2, 3, 2, 3, 0, 3, 2, 2, 2, 3, 1, 3, 0, 2, 0, 2, 2, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 1, 1, 2, 0, 1, 0, 1, 3]
[2 2 0 0 0 2 0 0 2 0 0 2 2 2 2 2 2 2 2 0 2 2 2 2 1 2 2 2 2 0 2 2 2 2 2 0 2
 2 2 0 0 0 0 2 2 2 2 0 0 0 2 1 0 0 0 0 2 2 2 1 2 2 2 0 2 2 2 2 2 2 2 2 0 2
 2 2 2 2 2 2 0 2 0 2 2 2 0 0 2 2 0 0 2 0 2 2 2 2 2 2 2 2 2 2 2]
Accuracy PCA + SVM: 0.42857142857142855


In [187]:
# PCA + RC
clf = RandomForestClassifier(n_estimators=100)
clf.fit(Pca_train_X,svm_y)
pred_rc=clf.predict(Pca_test_X)
score=metrics.accuracy_score(Label_OF_test_data, pred_rc)
print(Label_OF_test_data)
print(pred_rc)
print('Accuracy PCA + RANDOM CLASSIFIER: {}'.format(score))

[3, 2, 0, 1, 3, 1, 0, 1, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 2, 1, 1, 3, 2, 2, 1, 1, 3, 3, 2, 3, 0, 3, 1, 3, 3, 3, 0, 0, 3, 1, 2, 3, 1, 0, 3, 1, 1, 0, 0, 1, 0, 2, 0, 3, 1, 2, 3, 2, 0, 2, 2, 2, 2, 2, 3, 2, 3, 0, 3, 2, 2, 2, 3, 1, 3, 0, 2, 0, 2, 2, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 1, 1, 2, 0, 1, 0, 1, 3]
[2 2 0 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 0 2
 2 2 2 2 2 2 2 2 2 2 0 2 0 2 2 2 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 0 0 2 2 2 2 2 2 2 2 2 2]
Accuracy PCA + RANDOM CLASSIFIER: 0.29523809523809524


In [188]:
# PCA + KNN IMPLEMENTATION
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(Pca_train_X, svm_y)
pred_knn=neigh.predict(Pca_test_X)
score=metrics.accuracy_score(Label_OF_test_data, pred_knn)
print(Label_OF_test_data)
print(pred_knn)
print('Accuracy PCA + KNN : {}'.format(score))

[3, 2, 0, 1, 3, 1, 0, 1, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 2, 1, 1, 3, 2, 2, 1, 1, 3, 3, 2, 3, 0, 3, 1, 3, 3, 3, 0, 0, 3, 1, 2, 3, 1, 0, 3, 1, 1, 0, 0, 1, 0, 2, 0, 3, 1, 2, 3, 2, 0, 2, 2, 2, 2, 2, 3, 2, 3, 0, 3, 2, 2, 2, 3, 1, 3, 0, 2, 0, 2, 2, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 1, 1, 2, 0, 1, 0, 1, 3]
[2 2 0 1 2 2 0 0 2 0 0 2 2 2 2 2 2 2 2 0 2 2 0 2 2 2 2 2 2 1 2 2 2 2 2 0 2
 2 2 0 2 0 0 0 2 2 0 0 2 0 2 2 2 0 0 0 2 0 2 1 2 2 2 0 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 0 0 2 0 2 2 2 0 2 2 2 0 0 2 2 1 2 2 2 2 2 2 2 0 2 2]
Accuracy PCA + KNN : 0.4


In [189]:
# PCA + LinearDiscriminantAnalysis IMPLEMENTATION
clf = LinearDiscriminantAnalysis()
clf.fit(Pca_train_X, svm_y)
pred_LinearDiscriminantAnalysis=clf.predict(Pca_test_X)
score=metrics.accuracy_score(Label_OF_test_data, pred_LinearDiscriminantAnalysis)
print(Label_OF_test_data)
print(pred_LinearDiscriminantAnalysis)
print('Accuracy PCA + LinearDiscriminantAnalysis : {}'.format(score))


[3, 2, 0, 1, 3, 1, 0, 1, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 2, 1, 1, 3, 2, 2, 1, 1, 3, 3, 2, 3, 0, 3, 1, 3, 3, 3, 0, 0, 3, 1, 2, 3, 1, 0, 3, 1, 1, 0, 0, 1, 0, 2, 0, 3, 1, 2, 3, 2, 0, 2, 2, 2, 2, 2, 3, 2, 3, 0, 3, 2, 2, 2, 3, 1, 3, 0, 2, 0, 2, 2, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 1, 1, 2, 0, 1, 0, 1, 3]
[2 2 0 2 0 2 0 0 2 0 0 2 2 2 2 1 2 2 2 0 0 0 2 2 2 2 2 2 2 0 2 2 2 2 2 0 2
 2 2 0 2 0 0 2 2 0 2 2 0 0 2 1 2 0 0 0 2 0 2 1 2 2 2 0 2 2 2 2 2 2 2 2 0 0
 2 2 2 2 2 0 0 2 2 2 2 2 2 2 2 2 0 0 2 0 0 0 2 2 2 2 2 1 2 2 2]
Accuracy PCA + LinearDiscriminantAnalysis : 0.4095238095238095


In [190]:
# PCA + GaussianNB IMPLEMENTATION
clf = GaussianNB()
clf.fit(Pca_train_X, svm_y)
pred_GaussianNB=clf.predict(Pca_test_X)
score=metrics.accuracy_score(Label_OF_test_data, pred_GaussianNB)
print(Label_OF_test_data)
print(pred_GaussianNB)
print('Accuracy PCA + GaussianNB : {}'.format(score))

[3, 2, 0, 1, 3, 1, 0, 1, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 2, 1, 1, 3, 2, 2, 1, 1, 3, 3, 2, 3, 0, 3, 1, 3, 3, 3, 0, 0, 3, 1, 2, 3, 1, 0, 3, 1, 1, 0, 0, 1, 0, 2, 0, 3, 1, 2, 3, 2, 0, 2, 2, 2, 2, 2, 3, 2, 3, 0, 3, 2, 2, 2, 3, 1, 3, 0, 2, 0, 2, 2, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 1, 1, 2, 0, 1, 0, 1, 3]
[2 2 0 2 2 2 0 0 2 0 2 2 2 2 2 1 2 2 2 0 0 0 2 0 2 2 2 2 2 0 2 2 2 2 2 0 2
 2 2 0 0 2 0 2 2 0 2 0 0 0 2 1 2 0 2 0 2 0 2 1 2 2 0 2 2 2 2 2 2 2 2 2 0 0
 2 2 2 2 2 0 0 2 2 2 2 2 0 2 2 2 2 2 2 0 0 2 2 2 2 2 2 2 2 2 0]
Accuracy PCA + GaussianNB : 0.3523809523809524


In [191]:
# PCA + DecisionTreeClassifier 
clf = DecisionTreeClassifier(random_state=0)
clf.fit(Pca_train_X, svm_y)
pred_DecisionTreeClassifier=clf.predict(Pca_test_X)
score=metrics.accuracy_score(Label_OF_test_data, pred_DecisionTreeClassifier)
print(Label_OF_test_data)
print(pred_DecisionTreeClassifier)
print('Accuracy DecisionTreeClassifier : {}'.format(score))

[3, 2, 0, 1, 3, 1, 0, 1, 0, 0, 0, 0, 2, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 2, 1, 1, 3, 2, 2, 1, 1, 3, 3, 2, 3, 0, 3, 1, 3, 3, 3, 0, 0, 3, 1, 2, 3, 1, 0, 3, 1, 1, 0, 0, 1, 0, 2, 0, 3, 1, 2, 3, 2, 0, 2, 2, 2, 2, 2, 3, 2, 3, 0, 3, 2, 2, 2, 3, 1, 3, 0, 2, 0, 2, 2, 3, 1, 0, 0, 2, 1, 3, 3, 0, 0, 1, 2, 1, 1, 2, 0, 1, 0, 1, 3]
[1 2 0 0 2 2 0 2 0 2 0 0 2 2 2 0 2 2 2 0 2 0 2 0 2 2 2 2 2 0 2 0 2 2 2 0 2
 2 2 0 2 0 2 2 2 0 2 2 2 0 2 2 0 2 0 0 2 2 0 2 0 2 2 0 2 2 2 2 2 2 2 0 0 0
 2 2 2 2 2 0 0 2 2 1 2 2 0 0 2 2 0 0 2 0 0 2 2 2 2 0 2 2 1 2 2]
Accuracy DecisionTreeClassifier : 0.3333333333333333


# This is for LBP and Ml algo

In [192]:
from skimage import feature
# 
from skimage.feature import local_binary_pattern
from sklearn.svm import LinearSVC

In [193]:
# loading the training data
hist_train=[]
label_b4 = []
hist_train_label=[]
for img in tqdm(os.listdir(TRAIN_DIR)):

    # labeling the images
    label = label_img(img)
    label_b4.append(label)
    path = os.path.join(TRAIN_DIR, img)

    # loading the image from the path and then converting them into
    # greyscale for easier covnet prob
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    # resizing the image for processing them in the covnet
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    lbp = feature.local_binary_pattern(img, 24,8, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),
        bins=np.arange(0, 24+ 3),
        range=(0, 24+2))

    # normalize the histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    hist_train.append(hist)
        # final step-forming the training data list with numpy array of the images
#         training_data.append([np.array(img), np.array(label)])
 
#     # shuffling of the training data to preserve the random state of our data
#     shuffle(training_data)
 
#     # saving our trained data for further uses if required
#     np.save('train_data.npy', training_data)
hist_train[0:5]
for val in label_b4:
    if np.argmax(val) == 0:
#         str_label ='Happy'
        hist_train_label.append(0)
    elif np.argmax(val) == 1:
#         str_label ='sad'
        hist_train_label.append(1)
    elif np.argmax(val) == 2:
#         str_label ='Angry'
        hist_train_label.append(2)
    elif np.argmax(val) == 3:
#         str_label ='normal'
        hist_train_label.append(3)
hist_train_label
# svm_y[0:27]

100%|████████████████████████████████████████| 413/413 [00:04<00:00, 89.47it/s]


[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [194]:
model = LinearSVC(C=100.0, random_state=42)
model.fit(hist_train, hist_train_label)

LinearSVC(C=100.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

In [195]:
# loading testing data 
hist_test=[]
label_b4 = []
hist_test_label=[]
for img in tqdm(os.listdir(TEST_DIR)):

    # labeling the images
    label = label_img(img)
    label_b4.append(label)
    path = os.path.join(TEST_DIR, img)

    # loading the image from the path and then converting them into
    # greyscale for easier covnet prob
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    # resizing the image for processing them in the covnet
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    lbp = feature.local_binary_pattern(img, 24,8, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),
        bins=np.arange(0, 24+ 3),
        range=(0, 24+2))

    # normalize the histogram
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-7)
    hist_test.append(hist)
        # final step-forming the training data list with numpy array of the images
#         training_data.append([np.array(img), np.array(label)])
 
#     # shuffling of the training data to preserve the random state of our data
#     shuffle(training_data)
 
#     # saving our trained data for further uses if required
#     np.save('train_data.npy', training_data)
print(hist_test[0:5])
for val in label_b4:
    if np.argmax(val) == 0:
#         str_label ='Happy'
        hist_test_label.append(0)
    elif np.argmax(val) == 1:
#         str_label ='sad'
        hist_test_label.append(1)
    elif np.argmax(val) == 2:
#         str_label ='Angry'
        hist_test_label.append(2)
    elif np.argmax(val) == 3:
#         str_label ='Angry'
        hist_test_label.append(3)
hist_test_label

100%|████████████████████████████████████████| 105/105 [00:01<00:00, 87.58it/s]


[array([0.0704, 0.0412, 0.0384, 0.0308, 0.0176, 0.0124, 0.0056, 0.0052,
       0.0048, 0.0068, 0.012 , 0.0204, 0.0188, 0.0224, 0.012 , 0.0132,
       0.0032, 0.0072, 0.0076, 0.0076, 0.0056, 0.006 , 0.0072, 0.012 ,
       0.0172, 0.5944]), array([0.0524, 0.0528, 0.054 , 0.0312, 0.0144, 0.016 , 0.0096, 0.0032,
       0.0056, 0.008 , 0.01  , 0.006 , 0.0088, 0.0156, 0.0088, 0.0048,
       0.0088, 0.0084, 0.0124, 0.0112, 0.0076, 0.008 , 0.006 , 0.0156,
       0.0256, 0.5952]), array([0.062 , 0.0468, 0.0224, 0.0224, 0.0204, 0.012 , 0.0084, 0.008 ,
       0.0124, 0.0196, 0.0256, 0.018 , 0.016 , 0.018 , 0.0116, 0.0084,
       0.004 , 0.01  , 0.0072, 0.008 , 0.004 , 0.006 , 0.0096, 0.0136,
       0.0136, 0.592 ]), array([0.05  , 0.0492, 0.034 , 0.0424, 0.0176, 0.0148, 0.0068, 0.0072,
       0.0128, 0.0184, 0.0192, 0.0068, 0.0088, 0.0132, 0.0112, 0.014 ,
       0.0112, 0.0064, 0.008 , 0.006 , 0.0064, 0.006 , 0.0088, 0.016 ,
       0.02  , 0.5848]), array([0.0876, 0.0432, 0.0544, 0.0284, 0.0232, 

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [196]:
prediction = model.predict(hist_test)

score=metrics.accuracy_score(hist_test_label, prediction)
print(hist_test_label)
print(prediction)
print('Accuracy LinearSVC: {}'.format(score))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 0 2 2 2 0 2 0
 2 0 2 2 2 0 0 2 2 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2]
Accuracy LinearSVC: 0.6190476190476191


In [197]:
# LBP + RC
clf = RandomForestClassifier(n_estimators=100)
clf.fit(hist_train,hist_train_label)
pred_rc=clf.predict(hist_test)
score=metrics.accuracy_score(hist_test_label, pred_rc)
print(hist_test_label)
print(pred_rc)
print('Accuracy LBP + RANDOM CLASSIFIER: {}'.format(score))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0 2 2 0 0 2 2 0 2 0
 2 0 2 0 2 0 0 2 2 0 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 0 2 2 0 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2]
Accuracy LBP + RANDOM CLASSIFIER: 0.6285714285714286


In [198]:
# LBP + KNN IMPLEMENTATION
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(hist_train, hist_train_label)
pred_knn=neigh.predict(hist_test)
score=metrics.accuracy_score(hist_test_label, pred_knn)
print(hist_test_label)
print(pred_knn)
print('Accuracy LBP + KNN : {}'.format(score))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0 0 2 2 2 1 1 2 0 2
 2 1 0 0 2 0 2 2 0 2 0 2 2 2 2 2 2 2 2 2 2 0 0 2 2 2 0 0 2 2 2 2 2 2 2 2 2
 2 2 2 2 0 2 2 2 0 2 1 1 0 2 2 0 0 2 2 2 0 2 2 2 2 2 2 2 2 2 0]
Accuracy LBP + KNN : 0.580952380952381


In [199]:
# LBP + LinearDiscriminantAnalysis IMPLEMENTATION
clf = LinearDiscriminantAnalysis()
clf.fit(hist_train, hist_train_label)
pred_LinearDiscriminantAnalysis=clf.predict(hist_test)
score=metrics.accuracy_score(hist_test_label, pred_LinearDiscriminantAnalysis)
print(hist_test_label)
print(pred_LinearDiscriminantAnalysis)
print('Accuracy LBP + LinearDiscriminantAnalysis : {}'.format(score))

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 0 2 2 2 0 2 0
 2 0 2 2 2 0 0 2 2 0 0 2 0 2 0 2 2 2 1 2 2 0 2 2 2 2 2 2 2 0 0 2 2 2 2 2 2
 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 2 0 2 2 0 2 2 2 2 2 2 2]
Accuracy LBP + LinearDiscriminantAnalysis : 0.5523809523809524


In [200]:
# LBP + GaussianNB IMPLEMENTATION
clf = GaussianNB()
clf.fit(hist_train, hist_train_label)
pred_GaussianNB=clf.predict(hist_test)
score=metrics.accuracy_score(hist_test_label, pred_GaussianNB)
print(hist_test_label)
print(pred_GaussianNB)
print('Accuracy LBP + GaussianNB : {}'.format(score))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 0 2 2 0 0 2 0 0 0 2
 2 0 0 2 0 0 0 0 2 0 0 2 2 2 0 0 2 2 1 0 2 2 2 2 2 0 2 0 0 0 0 2 2 2 2 0 0
 0 2 2 0 2 0 2 1 2 0 2 2 2 2 2 2 2 2 2 0 0 0 2 2 0 2 2 0 2 2 2]
Accuracy LBP + GaussianNB : 0.5333333333333333


In [201]:
# LBP + DecisionTreeClassifier 
clf = DecisionTreeClassifier(random_state=0)
clf.fit(hist_train, hist_train_label)
pred_DecisionTreeClassifier=clf.predict(hist_test)
score=metrics.accuracy_score(hist_test_label, pred_DecisionTreeClassifier)
print(hist_test_label)
print(pred_DecisionTreeClassifier)
print('Accuracy DecisionTreeClassifier : {}'.format(score))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[2 2 2 2 0 2 2 2 0 2 2 2 1 2 2 2 2 2 2 1 2 1 2 0 2 2 0 2 2 0 0 0 0 2 0 2 0
 2 2 2 0 2 0 0 2 0 2 0 2 2 2 2 0 2 0 2 2 2 0 2 2 0 2 2 2 2 2 1 0 2 2 0 2 2
 0 0 0 2 2 2 0 2 2 2 2 0 2 0 2 2 0 2 2 0 0 2 2 2 2 1 2 1 2 2 1]
Accuracy DecisionTreeClassifier : 0.5238095238095238
